In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [2]:
# Set the system Path Constants
# 1. Meter Event Data
RAW_DATA_DIR = "Data\\RAW"
EVENT_DIR = "EVENT_DATA"
OUTPUT_DIR = "Data\\PROCESSED"
# 2. Load Survey Data of Meters . Load Consumption Reported every 30 mins
LOADSURVEY_DIR = "LOAD_SURVEY"
# 3. User electricity consumption data
CONSUMPTION_DATAFILE = "ConsumptionData_2014_2017.xlsx"
# 4. Customer Master Table Data
CUSTOMERMASTER_DATAFILE = "CustomerMasterData.xlsx"
# 5. Meter Reading data
METERREADING_DATAFILE = "MeterReading_2014_2017.xlsx"
# 6. Sanctioned Load history of Users
SANCTIONLOAD_HISTORY_DATAFILE = "SanctionLoadHistory.xlsx"
# 7. Master table of location/area information
AREA_MASTER_DATAFILE = "Area Master.xlsx"
# 8. Need confirmation from NPCL
ASSESSED_UNITS_DATAFILE = "Assessed Units.xlsx"
# 9. Meter Replacement Data
METER_REPLACEMENTS_DATAFILE = "Meter Replacements.xlsx"

RATE_CATEGORY_FILE = "RateCategoryInfo.xlsx"
# 10. Meter Testing Cases supplied by Testing
#METER_TESTING_DATAFILE = "Top 3000 Cases at 440V_Load History Since 01.04.2014 updated.xlsx"
# 11. Meter Master 
#METER_MASTER = "Meter Master.xlsx"

In [3]:
# 1. Load Event Data-set
first = True
#filenames = [os.path.join(rt,file) for rt, sdir, files in os.walk(EVENT_DIR) for file in files \
#             if file.endswith('.xlsx') or file.endswith('.xls')]
#EventDataMasterDF = [pd.read_excel(file) for root,subdirs, files in os.walk(EVENT_DIR) for file in files]
for root, subdirs, files in os.walk(os.path.join(RAW_DATA_DIR,EVENT_DIR)):
    for file in files:
        #print(os.path.join(root,file))
        fpath = os.path.join(root,file)
        if first == True:
            EventMasterDF = pd.read_csv(fpath, sep = "\t")
            EventMasterDF = EventMasterDF.loc[:,~EventMasterDF.columns.str.contains('^Unnamed')]
            #print(fpath,EventMasterDF.dtypes)
            first = False
        else:
            tempDF = pd.read_csv(fpath, sep = "\t")
            tempDF = tempDF.loc[:,~tempDF.columns.str.contains('^Unnamed')]
            EventMasterDF = EventMasterDF.append(tempDF, ignore_index = True)
            #EventMasterDF = pd.read_excel(fpath)
            #print(fpath,EventMasterDF.dtypes)
# Convert the date and time into correct format
EventMasterDF['DATETIMESTAMP'] = EventMasterDF['EVENTDATE'].str.strip() + ' ' + EventMasterDF['EVENTTIME'].str.strip()
EventMasterDF['DATETIMESTAMP'] = pd.to_datetime(EventMasterDF.DATETIMESTAMP, format = '%d-%b-%Y %H:%M %p')
#EventMasterDF.EVENTDATE = pd.to_datetime(EventMasterDF.EVENTDATE, format = '%d-%b-%Y')

# Select the rows with suspicious events
# Priority of events is given below(From Highest to Lowest):
# 1. Current Without Voltage (1(L1),2(L2),3(L3))
# 2. Current Terminal Short (19)
# 2. CT Bypass / Loss of Neutral (1201)
# 3. Current Terminal Open (23,24,25,26)
# 4. Current missing (30,31,32)
# 5. Voltage Imbalance (14)
# 6. Invalid Phase association/Invalid Voltage (18,66)
# 7. Neural Disturbance (28)
# 7. Loss of Neutral (1208)
# 8. Magnet Tamper (27)
# 9. Cover Open (60)


suspicious_events = [1,2,3,14,18,19,23,24,25,26,27,28,30,31,32,60,66,1201,1208]
SuspiciousEventDF = EventMasterDF.loc[EventMasterDF['CODE'].isin(suspicious_events)]
SuspiciousEventDF = SuspiciousEventDF.sort_values(['METERNO','CODE','DATETIMESTAMP'],ascending=[True,True,True])

#print(SuspiciousEventDF.dtypes)
#print(SuspiciousEventDF.head(10))


# 2. Read Load Survey data-set of consumers
# The Load Survey Data contains following fields
# Consumer Feederid, MeterNumber, Date, Time, Parameter Code, Parameter Type, Parameter Description,
# Value and Unit.
# Parameter Codes
# 1. P7-1-18-1-0 : Absolute Active Consumption (in kWh)
# 2. P7-3-18-2-0 : Absolute Apparent Consumption (in kVAh)
# 3. P7-2-9-2-0 : Absolute Reactive Lag (in kVArh)
# 4. P7-2-10-2-0 : Absolute Reactive lead (in kVArh)
# 5. P1-2-1-4-0 : Voltage on L1
# 6. P1-2-2-4-0 : Voltage on L2
# 7. P1-2-3-4-0 : Voltage on L3
# 8. P2-1-1-4-0 : Current on L1
# 9. P2-1-2-4-0 : Current on L2
# 10. P2-1-3-4-0 : Current on L3

select_cols = ["CONSUMER_FEEDERID","METERNO","LSDATE","LSTIME","PARAM_CODE","VALUE"]
select_params = ['P7-1-18-1-0','P7-3-18-2-0','P1-2-1-4-0','P1-2-2-4-0','P1-2-3-4-0','P2-1-1-4-0',
                'P2-1-2-4-0','P2-1-3-4-0']
first = True

for root, subdirs, files in os.walk(os.path.join(RAW_DATA_DIR,LOADSURVEY_DIR)):
    for file in files:
        #print(os.path.join(root,file))
        fpath = os.path.join(root,file)
        # Take the load survey data of year 2016 and 2017.
        #year = int(os.path.basename(os.path.dirname(fpath)))
        #if year < 2016:
        #    continue
        LoadSurveyRaw = pd.read_csv(fpath, sep = "\t")
        LoadSurveyRaw = LoadSurveyRaw.loc[:,~LoadSurveyRaw.columns.str.contains('^Unnamed')]
        LoadSurveyRaw = LoadSurveyRaw[select_cols]
        
        # Select only the rows with desired parameters
        LoadSurveyRaw = LoadSurveyRaw.loc[LoadSurveyRaw['PARAM_CODE'].isin(select_params)]
        LoadSurveyRaw['DATETIMESTAMP'] = LoadSurveyRaw['LSDATE'].str.strip() + ' ' + LoadSurveyRaw['LSTIME'].str.strip()
        LoadSurveyRaw['DATETIMESTAMP'] = pd.to_datetime(LoadSurveyRaw.DATETIMESTAMP, format = '%d-%b-%Y %H:%M %p')
                
        if first == True:
            LoadSurveyMasterDF = LoadSurveyRaw
            first = False
            #print(fpath, LoadSurveyMasterDF.dtypes)
        else:
            LoadSurveyMasterDF = LoadSurveyMasterDF.append(LoadSurveyRaw, ignore_index = True)
            #print(fpath, tempDF.dtypes)
            #del(tempDF)
LoadSurveyMasterDF = LoadSurveyMasterDF.drop_duplicates()
# Convert the LSDATE into date format and LTIME into time format
# Converted formats are yyyy-mm-dd
#LoadSurveyMasterDF.LSDATE = pd.to_datetime(LoadSurveyMasterDF.LSDATE, format = "%d-%b-%Y")
#print(LoadSurveyMasterDF.dtypes)

In [4]:
# 3. Read consumption Data
ConsumptionMasterDF = pd.read_excel(os.path.join(RAW_DATA_DIR,CONSUMPTION_DATAFILE))

# Rename the variables
NewVariableNames = ['InvoiceNumber','BillDocNumber','MonthofConsumption','ConsumerNumber','HouseNumber','Street','CareOf',\
                    'FullName','ConsumptionFrom','ConsumptionTo','BillingPortion','RateCategory','BillPostingDate','BillPostingMonth',\
                    'AccountClass','ContractID','RKey','PAccClass','GLFlag','ReverseInvoiceFlag','Unit','CreationDate','UOM',\
                   'BillingFactor','VAL_INT','BillableDemandConsumption','ISU_Consumption_KVAH','ISU_Consumption_KWH','PenalDemand',\
                   'RecordedDemand','Consumption_TOD1','Consumption_TOD2','Consumption_TOD3','LoadFactor','PowerFactor','Voltage']

# TOD : Time of Difference
# TOD1 : 22:00 Hrs to 06:00 Hrs -> Below stated rates
# TOD2 : 06:00 Hrs to 17:00 Hrs -> Same as stated rates
# TOD3 : 17:00 Hrs to 22:00 Hrs -> Above the stated rates
ConsumptionMasterDF.columns = NewVariableNames
#print(ConsumptionMasterDF.dtypes)
# drop the duplicate entries
ConsumptionMasterDF = ConsumptionMasterDF.drop_duplicates()

# Convert the Date variables into appropriate formats
ConsumptionMasterDF.MonthofConsumption = pd.to_datetime(ConsumptionMasterDF.MonthofConsumption, format = '%Y%m')
ConsumptionMasterDF.ConsumptionFrom = pd.to_datetime(ConsumptionMasterDF.ConsumptionFrom, format = '%d.%m.%Y')
ConsumptionMasterDF.ConsumptionTo = pd.to_datetime(ConsumptionMasterDF.ConsumptionTo, format = '%d.%m.%Y')
ConsumptionMasterDF.BillPostingDate = pd.to_datetime(ConsumptionMasterDF.BillPostingDate, format = '%d.%m%Y')
ConsumptionMasterDF.BillPostingMonth = pd.to_datetime(ConsumptionMasterDF.BillPostingMonth, format = '%m%Y')
ConsumptionMasterDF.CreationDate = pd.to_datetime(ConsumptionMasterDF.CreationDate, format = '%d.%m.%Y')

#print(ConsumptionMasterDF.dtypes)
#ConsumptionMasterDF.head(10)

# 4. Load CustomerMaster Data
CustomerMasterDF = pd.read_excel(os.path.join(RAW_DATA_DIR,CUSTOMERMASTER_DATAFILE))
#print(CustomerMasterDF.dtypes)

# Rename the variables
NewVariableNames = ['ConsumerNumber','ContractAcct','Contract','Installation','Name','CareOf','Street','HouseNo','TelephoneNumber',\
                   'MRU','MRUText','Portion','PoleNumber','RateCategory','Load','LoadUoM','Voltage','Status','MoveInDate','MobileNumber',
                   'EmailID']
CustomerMasterDF.columns = NewVariableNames
#print(CustomerMasterDF.dtypes)

# Convert the Date variable to appropriate data-type
CustomerMasterDF.MoveInDate = pd.to_datetime(CustomerMasterDF.MoveInDate, format = '%Y-%m-%d', errors = 'ignore')

# 5. Read the Meter Reading Data-set
MeterReadingSheets = pd.ExcelFile(os.path.join(RAW_DATA_DIR,METERREADING_DATAFILE))
first = True
for sheet in MeterReadingSheets.sheet_names:
    if first == True :
        MeterReadingMasterDF = MeterReadingSheets.parse(sheet)
        first = False
    else:
        MeterReadingMasterDF = MeterReadingMasterDF.append(MeterReadingSheets.parse(sheet), ignore_index = True)


NewVariableNames = ['ConsumerNumber','Name','CareOf','HouseNumber','Street','RateCategory','BillingPortion','Equipment','MRU',\
                   'MeterReadingNote','ScheduledMRDate','ActualMRDate','Billed_KWH','Billed_KVRH','Billed_KVAH','Billed_KWH1',\
                   'Billed_KWH2','Billed_KWH3','Billed_KAVH1','Billed_KVAH2','Billed_KVAH3','Billed_KVA1','Billed_KVA2',\
                   'Billed_KVA3','Billed_KVA']

# Replace the variables with new variables
MeterReadingMasterDF.columns = NewVariableNames

# Convert Date Variables in appropriate format
MeterReadingMasterDF.ScheduledMRDate = pd.to_datetime(MeterReadingMasterDF.ScheduledMRDate, format = '%m/%d/%Y', errors = 'ignore')
MeterReadingMasterDF.ActualMRDate = pd.to_datetime(MeterReadingMasterDF.ActualMRDate, format = '%m/%d/%Y', errors = 'ignore')

#print(MeterReadingMasterDF.dtypes)
#print(MeterReadingMasterDF.head(10))

# 6. Load Sanctioned Load History Data-set
SanctionedLoadHistoryDF = pd.read_excel(os.path.join(RAW_DATA_DIR,SANCTIONLOAD_HISTORY_DATAFILE))

# Rename the variables
NewVariableNames = ['ConsumerNumber','ContractAcc','ConsumerName','HouseNumber','Street','RateCategory','Voltage','Installation',\
                   'LoadUOM','SanctionedLoad','ValidFromDate','ValidToDate']
SanctionedLoadHistoryDF.columns = NewVariableNames

# Convert Date Variables in appropraite Date format
SanctionedLoadHistoryDF.ValidFromDate = pd.to_datetime(SanctionedLoadHistoryDF.ValidFromDate, format = '%d.%m.%Y', errors = 'ignore')

# Add the handler for OutofBound error Dates. Replace 31.12.9999 with 31.12.2050
SanctionedLoadHistoryDF.loc[SanctionedLoadHistoryDF['ValidToDate'] == '31.12.9999',['ValidToDate']]= '31.12.2050'
SanctionedLoadHistoryDF.ValidToDate = pd.to_datetime(SanctionedLoadHistoryDF.ValidToDate, format = '%d.%m.%Y')

#print(SanctionedLoadHistoryDF.dtypes)
#print(SanctionedLoadHistoryDF.head(10))

# 7. Load Area Master Data
AreaMasterDF = pd.read_excel(os.path.join(RAW_DATA_DIR,AREA_MASTER_DATAFILE))
# Create New variable names
NewVariableNames = ['StreetNumber','CityNumber','District','StreetTypeNumber','StreetType','RegSTGrp','TaxJuris','Street',\
                    'streetabbr','streetabbr15']
AreaMasterDF.columns = NewVariableNames

#print(AreaMasterDF.dtypes)
#print(AreaMasterDF.head(10))

# 8. Meter Assesment Data
MeterAssessmentMasterDF = pd.read_excel(os.path.join(RAW_DATA_DIR,ASSESSED_UNITS_DATAFILE))

NewVariableNames = ['ConsumerNumber','AssessedUnitsinKWH','PostingDate']
MeterAssessmentMasterDF.columns = NewVariableNames

# Convert date into propter date format
MeterAssessmentMasterDF.PostingDate = pd.to_datetime(MeterAssessmentMasterDF.PostingDate, format = '%d.%m.%Y')
#print(MeterAssessmentMasterDF.dtypes)
#print(MeterAssessmentMasterDF.head(10))

# 9. Read Meter Replacement Data
MeterReplacementMasterDF = pd.read_excel(os.path.join(RAW_DATA_DIR,METER_REPLACEMENTS_DATAFILE))

# Create New Variables
NewVariableNames = ['ConsumerNumber','InstallationNumber','EquipmentID','ManufacturerSerialNumber','BillingFactor','ValidFrom',\
                   'ValidTo']
MeterReplacementMasterDF.columns = NewVariableNames

# Convert Date into proper format
#MeterReplacementMasterDF.ValidFrom = pd.datetime(MeterReplacementMasterDF.ValidFrom, format = "%d-%m-%Y")
#MeterReplacementMasterDF.ValidTo = pd.datetime(MeterReplacementMasterDF.ValidTo, format = "%d-%m-%Y %hh:%mm:%ss")

#print(MeterReplacementMasterDF.dtypes)
#print(MeterReplacementMasterDF.head(10))

# 10. Read Meter Testing 
#MeterTestingMasterDF = pd.read_excel(os.path.join(RAW_DATA_DIR,METER_TESTING_DATAFILE))
#NewVariablesCols = ['SL','ServiceOrderNumber','ReferenceDate','ConsumerNumber','AccuracyPercentageKWH','ContractAcc',\
#                   'Contract','Installation','Name','CareOf','Street','HouseNumber','TelephoneNumber','MRU','MRUText',\
#                   'BillingPortion','PoleNumber','RateCategory','Load','LoadUoM','Voltage','Status','MoveInDate','MobileNumber',\
#                   'EmailID']
#MeterTestingMasterDF.columns = NewVariablesCols
#print(MeterTestingMasterDF.dtypes)

#MeterMasterDF = pd.read_excel(os.path.join(RAW_DATA_DIR,METER_MASTER))
#MeterMasterDF.columns = ['ConsumerNumber','METERNO','SerialNumber','Manufacturer','RateCategory','MeterStatus','BillingPortion',\
#                        'MRUnit','Installation','BillingFactor','ValidFrom','ValidTo']

# Extract only the revelant fields with valid explanations
# 1. In CustomerMasterDF <ConsumerNumber, Name, Street, RateCategory, Load, LoadUoM, Voltage, Status, MoveInDate>
CustomerDF = CustomerMasterDF[["ConsumerNumber", "Name", "Street", "RateCategory", "Load", "LoadUoM", "Voltage", \
                               "Status", "MoveInDate"]]
# 2. Extract only relevant fields from laod survey data
#LoadSurveyDF = LoadSurveyMasterDF

# 3. Extract Event Data
#EventDF = EventMasterDF

# 4. ConsumptionMasterDF
SelectCols = ['InvoiceNumber','BillDocNumber','ConsumerNumber','MonthofConsumption','Street','ConsumptionFrom','ConsumptionTo','RateCategory',\
              'ReverseInvoiceFlag','CreationDate','UOM','BillingFactor','BillableDemandConsumption','ISU_Consumption_KVAH',\
              'ISU_Consumption_KWH','PenalDemand','RecordedDemand','Consumption_TOD1','Consumption_TOD2','Consumption_TOD3',\
              'LoadFactor','PowerFactor','Voltage']
ConsumptionDF = ConsumptionMasterDF[SelectCols]

# 5. MeterReadingMasterDF
SelectCols = ['ConsumerNumber','Street','RateCategory','MeterReadingNote','ActualMRDate',\
              'Billed_KWH','Billed_KVRH','Billed_KVAH','Billed_KWH1','Billed_KWH2','Billed_KWH3','Billed_KAVH1',\
              'Billed_KVAH2','Billed_KVAH3','Billed_KVA1','Billed_KVA2','Billed_KVA3','Billed_KVA']
MeterReadingDF = MeterReadingMasterDF[SelectCols]

# 6.SanctionedLoadHistoryDF
SelectCols = ['ConsumerNumber','Street','RateCategory','Voltage','LoadUOM','SanctionedLoad','ValidFromDate','ValidToDate']
SanctionedLHDF = SanctionedLoadHistoryDF[SelectCols]

# 7. Area Master
Selectcols = ['StreetTypeNumber','Street','StreetType']
StreetMappingDF = AreaMasterDF[Selectcols]

# 8. MeterReplacementMasterDF
Selectcols = ['ConsumerNumber','EquipmentID','BillingFactor','ValidFrom','ValidTo']
MeterReplacementDF = MeterReplacementMasterDF[Selectcols]

# 9. MeterTestingMasterDF
#MeterTestingDF = MeterTestingMasterDF[['SL','ServiceOrderNumber','ReferenceDate','ConsumerNumber','AccuracyPercentageKWH',\
#                                       'Street','RateCategory','Load','LoadUoM','Voltage','Status','MoveInDate']]

# 10. MeterMasterDF
#MeterMasterDF = MeterMasterDF[['ConsumerNumber','METERNO','ValidFrom','ValidTo']]

RateCategoryInfo = pd.read_excel(os.path.join(RAW_DATA_DIR,RATE_CATEGORY_FILE))
#
CustomerWithStreetTypeDF = pd.merge(CustomerDF,StreetMappingDF,how='left',on='Street')
CustomerWithStreetTypeDF = pd.merge(CustomerWithStreetTypeDF,RateCategoryInfo, how = 'left', on = 'RateCategory')
#

# Save as csv files 
CustomerWithStreetTypeDF.to_csv(os.path.join(OUTPUT_DIR,"CustomerMasterDF.csv"),index=False)
#LoadSurveyDF.to_csv(os.path.join(OUPUT_DIR,"LoadSurveyDF.csv"),index=False)
#EventDF.to_csv(os.path.join(OUPUT_DIR,"EventDF.csv"),index=False)
ConsumptionDF.to_csv(os.path.join(OUTPUT_DIR,"ConsumptionDF.csv"),index=False)
MeterReadingDF.to_csv(os.path.join(OUTPUT_DIR,"MeterReadingDF.csv"),index=False)
SanctionedLHDF.to_csv(os.path.join(OUTPUT_DIR,"SanctionedLHDF.csv"),index=False)
StreetMappingDF.to_csv(os.path.join(OUTPUT_DIR,"StreetMappingDF.csv"),index=False)
MeterReplacementDF.to_csv(os.path.join(OUTPUT_DIR,"MeterReplacementDF.csv"),index=False)
SuspiciousEventDF.to_csv(os.path.join(OUTPUT_DIR,"SuspiciousEventDF.csv"), index=False)
#MeterTestingDF.to_csv(os.path.join(OUPUT_DIR,"MeterTestingDF.csv"),index=False)
print('completed')

completed
